# Trace Cross Encoder

In [1]:
import torch
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_id = "cross-encoder/stsb-roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)
features = tokenizer([('Query', 'Paragraph1'), ('Query', 'Paragraph2') , ('Query', 'Paragraph3')],  padding=True, truncation=True, return_tensors="pt")

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
features

{'input_ids': tensor([[    0, 48382,     2,     2, 22011, 44947,   134,     2],
        [    0, 48382,     2,     2, 22011, 44947,   176,     2],
        [    0, 48382,     2,     2, 22011, 44947,   246,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [4]:
#ex_input = (features["input_ids"], features["token_type_ids"], features["attention_mask"])
ex_input = (features["input_ids"], features["attention_mask"])
traced_model = torch.jit.trace(model, ex_input, strict=False)
torch.jit.save(traced_model, "traced_cross_encoder.pt")

In [5]:
loaded_model = torch.jit.load("traced_cross_encoder.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=RobertaForSequenceClassification
  (roberta): RecursiveScriptModule(
    original_name=RobertaModel
    (embeddings): RecursiveScriptModule(
      original_name=RobertaEmbeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (encoder): RecursiveScriptModule(
      original_name=RobertaEncoder
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=RobertaLayer
          (attention): RecursiveScriptModule(
            original_name=RobertaAttention
            (self): RecursiveScriptModule(
              original_name=RobertaSelfAttention
              (query): R

In [6]:
test_sentences = [('How many people live in Berlin?', 'How many people live in Berlin?'), ('Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.')]
test_features = tokenizer(test_sentences,  padding=True, truncation=True, return_tensors="pt")
pt_prediction = loaded_model(**test_features)
pt_prediction

{'logits': tensor([[ 5.6635],
         [-3.7582]], grad_fn=<AddmmBackward0>)}

# Compare Output

In [7]:
from sentence_transformers import CrossEncoder
model = CrossEncoder(model_id)
original_embedding = model.predict(test_sentences)
original_embedding

array([0.99654156, 0.02279299], dtype=float32)

In [54]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

features = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

tensor([[ 5.6635],
        [-3.7582]])


In [66]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, PreTrainedTokenizerFast
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = PreTrainedTokenizerFast(tokenizer_file="cross-encoder-torchscript/stsb-TinyBERT-L-4/tokenizer.json")
tokenizer.add_special_tokens({
  "cls_token": "[CLS]",
  "mask_token": "[MASK]",
  "pad_token": "[PAD]",
  "sep_token": "[SEP]",
  "unk_token": "[UNK]"
}
)
features = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")
model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


IndexError: index out of range in self